In [3]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.6 kB/s eta 0:00:00m eta 0:00:010:00:08m
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.3 kB/s eta 0:00:00m eta 0:00:010:00:04m
  Using cached ml_dtypes-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

input_shape = (224, 224, 3)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [16]:
train_dir = '/home/sahil/Facial Recognition Model/Train'
validation_dir = '/home/sahil/Facial Recognition Model/Validation'



train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 147 images belonging to 5 classes.
Found 21 images belonging to 5 classes.


In [17]:
# Build the model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,574,853 (13.64 MB)

 Trainable params: 1,316,869 (5.02 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [18]:

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('model.keras', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
]


history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks
)


Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 859ms/step - accuracy: 0.3263 - loss: 2.0503 - val_accuracy: 0.6190 - val_loss: 1.8708
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 607ms/step - accuracy: 0.5735 - loss: 1.8598 - val_accuracy: 0.7143 - val_loss: 0.8943
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 636ms/step - accuracy: 0.7831 - loss: 0.6831 - val_accuracy: 0.7143 - val_loss: 0.6658
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 630ms/step - accuracy: 0.8904 - loss: 0.4505 - val_accuracy: 0.8571 - val_loss: 0.5914
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 624ms/step - accuracy: 0.9012 - loss: 0.3278 - val_accuracy: 0.7619 - val_loss: 0.5309
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 659ms/step - accuracy: 0.9297 - loss: 0.2745 - val_accuracy: 0.8571 - val_loss: 0.2425
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 595ms/step - accuracy: 0.8903 - loss: 0.2890 - val_accuracy: 0.9048 - val_loss: 0.2834
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 533ms/step - accuracy: 0.9096 - loss: 0.2842 - val_accuracy: 0.7619 - val_loss:

In [19]:

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')

# Save the model
model.save('face_recognition_model.keras')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step - accuracy: 0.9524 - loss: 0.1610
Validation accuracy: 95.24%
